In [1]:
from simpler import load

data = load('12.txt')
board = data.strip().split('\n')

In [2]:
NEIGHBORS = (-1, 0), (0, -1), (0, 1), (1, 0)

cells = {}  # (r, c): (region, [up_wall?, left_wall?, right_wall?, down_wall?])
regions = {}  # region: [(r, c), ...]

# assign each cell its own region and find its walls
for r in range(len(board)):
	for c in range(len(board[0])):
		region = r * len(board[0]) + c
		walls = []
		for r_off, c_off in NEIGHBORS:
			nr, nc = r + r_off, c + c_off
			walls.append(nr == -1 or nc == -1 or nr == len(board) or nc == len(board[0]) or board[nr][nc] != board[r][c])
		cells[r, c] = [region, walls]
		regions[region] = [(r, c)]

# merge regions with the same crop
for r in range(len(board)):
	for c in range(len(board[0])):
		id = cells[r, c][0]
		for r_off, c_off in NEIGHBORS[:2]:
			nr, nc = r + r_off, c + c_off
			if nr < 0 or nc < 0: continue
			nid = cells[nr, nc][0]
			if board[r][c] == board[nr][nc] and id != nid:
				new_id, old_id = min(id, nid), max(id, nid)
				for cell in regions[old_id]: cells[cell][0] = new_id
				regions[new_id] += regions[old_id]
				del regions[old_id]
				id = new_id

sum(len(cs) * sum(sum(cells[c][1]) for c in cs) for cs in regions.values())

1461806

In [3]:
for region, cs in regions.items():
	prev = None
	for cell in sorted(cs):
		if prev is not None and prev[0] == cell[0] and prev[1] == cell[1] - 1:  # if the previous was at the left
			if cells[cell][1][0] and cells[prev][1][0] is not False:
				cells[cell][1][0] = None  # if it had an upper wall, this one is part of the same wall
			if cells[cell][1][3] and cells[prev][1][3] is not False:
				cells[cell][1][3] = None  # if it had a lower wall, the previous one is part of the same wall
		prev = cell
	prev = None
	for cell in sorted(cs, key=lambda c: (c[1], c[0])):
		if prev is not None and prev[0] == cell[0] - 1 and prev[1] == cell[1]:  # if the previous was at the top
			if cells[cell][1][1] and cells[prev][1][1] is not False:
				cells[cell][1][1] = None  # if it had a left wall, this one is part of the same wall
			if cells[cell][1][2] and cells[prev][1][2] is not False:
				cells[cell][1][2] = None  # if it had a right wall, the previous one is part of the same wall
		prev = cell

sum(len(cs) * sum(sum(1 for w in cells[c][1] if w) for c in cs) for cs in regions.values())

887932